In [1]:
from __future__ import print_function, division
from neuralnilm.source import RandomSegments, RandomSegmentsInMemory, SameLocation
import matplotlib.pyplot as plt
import numpy as np
from sys import stdout

In [2]:
source = SameLocation(
    filename='/data/mine/vadeec/merged/ukdale.h5',
    target_appliance='fridge freezer',
    window=("2013-03-18", "2013-04-18"),
    seq_length=512,
    train_buildings=[1],
    validation_buildings=[1],
    n_seq_per_batch=8,
    standardise_input=True,
    standardise_targets=True,
    offset_probability=0.5,
    ignore_offset_activations=True
)

Loaded 759 activations.
Loaded mains data for building 1


In [3]:
def plot_data(X, y, seq_i=0):
    fig, axes = plt.subplots(nrows=2, sharex=True)
    axes[0].plot(y[seq_i, :, 0])
    axes[1].plot(X[seq_i, :, 0])
    plt.show()

In [15]:
X_val, y_val = source.validation_data()

In [13]:
plot_data(X_val, y_val, 0)

In [6]:
source.start()

In [10]:
X, y = source.queue.get(timeout=5)

In [ ]:
i = 0
while True:
    print("\r", i, end="")
    stdout.flush()
    X, y = source.queue.get(timeout=5)
    if (X[0] == X_val[0]).all():
        print("ahhh!")
        break
    i += 1

 552